In [32]:
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons
from Ensemble import Ensemble, EnsembleMethods

In [33]:
model_path = '../models/swedish_leaf'
models_to_load = os.listdir(model_path)
models = dict(zip(list(map(lambda filename: filename.removesuffix('.h5'), models_to_load)),
                  list(map(lambda filename: keras.models.load_model(model_path + "/" + filename), models_to_load))))
models

/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


{'Encoder': <keras.engine.functional.Functional at 0x7f916e4a7970>,
 'FCN': <keras.engine.sequential.Sequential at 0x7f912c350970>,
 'MCDCNN': <keras.engine.sequential.Sequential at 0x7f90ec1ab070>,
 'MLP': <keras.engine.sequential.Sequential at 0x7f90e4463ee0>,
 'Resnet': <keras.engine.functional.Functional at 0x7f90e4476ef0>,
 'Time_CNN': <keras.engine.sequential.Sequential at 0x7f90e44b9ea0>}

In [24]:
ensemble_all = Ensemble(models=models.values(), ensemble_type=EnsembleMethods.AVERAGE)

In [30]:
ensemble_all(tf.random.normal((1,128)))
# TODO: actual evaluation

<tf.Tensor: shape=(15,), dtype=float32, numpy=
array([0.00543777, 0.40582752, 0.00788332, 0.0244417 , 0.0019312 ,
       0.1477799 , 0.00152094, 0.02781161, 0.17547236, 0.08840287,
       0.00077315, 0.08593228, 0.00071864, 0.02149367, 0.00457304],
      dtype=float32)>